In [2]:
import os
from fnmatch import fnmatch
from trainData_featExt_glcm import trainData_featExt_glcm
import numpy as np
from PIL import Image
from filtering.filters import Median
from feature_extraction.glcm import Glcm
from feature_extraction.gabor import Gabor
from sklearn import svm
import sklearn
import time
import pickle
from sklearn import preprocessing

In [ ]:
#Read features of training samples (30000x32)
train_samples_feats_matrix = np.load("train_samples_feats_matrix_Gabor.npy")

In [ ]:
#Train svn model from training data
t0 = time.time()
svm_model = svm.OneClassSVM(kernel='rbf', gamma=0.1, nu=0.1)
svm_model.fit(train_samples_feats_matrix)
print("traing time: ", time.time() - t0)

In [ ]:
print len(svm_model.support_vectors_)

In [ ]:
#load features matrix of validation dataset (10000x32)
valid_samples_feats_matrix_gabor = np.load("valid_samples_feats_matrix_Gabor.npy")

In [5]:
test_samples_feats_matrix_gabor = np.load("test_samples_feats_matrix_Gabor.npy")

In [3]:
svm_model = pickle.load( open( "SVM_Gabor_[0.2, 0.1]", "rb" ))

In [4]:
#Ground truth
y1 = np.repeat(+1, 6000) #normal
y2 = np.repeat(-1, 4000) #abnormal
y = np.concatenate((y1,y2))

In [ ]:
#Predict labels of training dataset
y_predict_train = svm_model.predict(train_samples_feats_matrix)
print ("training errors", y_predict_train[y_predict_train == -1].size)

In [5]:
#Predict labels of new data points
y_predict = svm_model.predict(test_samples_feats_matrix_gabor)
print ("number of samples detected as outliers", y_predict[y_predict == -1].size)
print ("detected as normal", y_predict[y_predict == +1].size)

('number of samples detected as outliers', 1816)
('detected as normal', 8184)


In [6]:
y_predict_score = svm_model.decision_function(test_samples_feats_matrix_gabor)

In [7]:
f1_binary = sklearn.metrics.f1_score(y, y_predict, pos_label =-1, average = 'binary')
f1_macro = sklearn.metrics.f1_score(y, y_predict, average = 'macro')
auc = sklearn.metrics.roc_auc_score(y, y_predict_score)
Math_Cof = sklearn.metrics.matthews_corrcoef(y, y_predict) 
tn, fp, fn, tp  = sklearn.metrics.confusion_matrix(y, y_predict).ravel()
print ("f1_binary: ", f1_binary, "f1_macro: ", f1_macro, "MAth_cof: ", Math_Cof , " auc: ", auc)
print("confusion matrix", tn, fp, fn, tp)

('f1_binary: ', 0.27888583218707014, 'f1_macro: ', 0.49160027649962645, 'MAth_cof: ', 0.044794429493101723, ' auc: ', 0.58371258333333331)
('confusion matrix', 811, 3189, 1005, 4995)


In [ ]:
import matplotlib.pyplot as pl
import matplotlib
from skimage import io

fpr,tpr,thresh = sklearn.metrics.roc_curve(y,y_predict_score)

f,ax = pl.subplots(1,1)
ax.plot(fpr,tpr,label="SVM")
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC curves")
ax.legend(loc="lower right")
# these are matplotlib.patch.Patch properties
props = dict(boxstyle='square', facecolor='white', alpha=0.5)

textstr = '$F1-binary=%.2f$\n$F1-macro=%.2f$\n$math-corcoeffient=%.2f$\n$AUC=%.2f$'%(0.64, 0.71, 0.44, 0.7)
# place a text box in upper left in axes coords
ax.text(0.95, 0.2, textstr, transform=ax.transAxes, fontsize=13,
    horizontalalignment='right', verticalalignment='bottom', bbox=props)

io.show()

In [ ]:
f.savefig('SVM_Gabor.png')